In [18]:
from bravado.client import SwaggerClient
gn = SwaggerClient.from_url('https://www.genomenexus.org/v2/api-docs',
                            config={"validate_requests":False,
                                    "validate_responses":False,
                                    "validate_swagger_spec":False})
# print(gn)

def getVariantInformation(s):
#     variant = gn.annotation_controlller.fetchVariantAnnotationByGenomicLocationGET(genomicLocation=s,
#     fields='hotspots mutation_assessor annotation_summary'.split()).result()
    variant = gn.annotation_controller.fetchVariantAnnotationByGenomicLocationGET(
        genomicLocation=s,
        # adds extra annotation resources, not included in default response:
        fields='hotspots mutation_assessor annotation_summary'.split()
    ).result()


    transcriptConsequences = variant.transcript_consequences
    extracted = {}
    for transcriptConsequence in transcriptConsequences:
        proteinPos = (transcriptConsequence['protein_start'], transcriptConsequence['protein_end'])
        if proteinPos is not (None,None):
            ensembleID = transcriptConsequence['transcript_id']
            extracted[ensembleID] = proteinPos
    return extracted

def getMatchedPTMs(ensembleID, proteinPos):
    ptms = gn.ptm_controller.fetchPostTranslationalModificationsGET(ensemblTranscriptId=ensembleID).result()
    matched = {}
    for ptm in ptms:
        pos = int(ptm['position'])
        if pos >= proteinPos[0] and pos <= proteinPos[1]:
            matched[pos] = ptm['type']
    return matched


def getPTMSFromLocation(loc):
    transcriptsToCheck = getVariantInformation(loc)
#     print(transcriptsToCheck)
    allMatchedPTMs = []
    for id in transcriptsToCheck.keys():
        matchedPTMs = getMatchedPTMs(id, transcriptsToCheck[id])
        if len(matchedPTMs) > 0 and matchedPTMs not in allMatchedPTMs:
            allMatchedPTMs.append(matchedPTMs)
    return allMatchedPTMs

import matplotlib.pyplot as plt
 
# set up the figure
fig = plt.figure()
ax = fig.add_subplot(111)
ax.set_xlim(0,10)
ax.set_ylim(0,10)
 
# draw lines
xmin = 0
xmax = 9
y = 5
height = 1
 
plt.hlines(y, xmin, xmax)
plt.vlines(xmin, y - height / 2., y + height / 2.)
plt.vlines(xmax, y - height / 2., y + height / 2.)
 
# draw a point on the line
px = 229/1000 * 9
plt.plot(px,y, 'ro', ms = 15, mfc = 'r')
 
# add an arrow
plt.annotate('Phosphorylation of PIK3CA - Residue 229', (px,y), xytext = (px - 1, y + 1),
              arrowprops=dict(facecolor='black', shrink=0.1),
              horizontalalignment='right')
 
# add numbers
plt.text(xmin - 0.1, y, '0', horizontalalignment='right')
plt.text(xmax + 0.1, y, '1000', horizontalalignment='left')
 
plt.axis('off')
plt.show()

In [17]:
maf_file = 'BigBoi' # Whatever MAP file is there
maf = open (maf_file, "r")
lines = maf.readlines()
maf.close()

for line in lines:
    #Get variant data from header
    if "Hugo_Symbol" not in line:
        edit = line.split("\t")
        chromNum = edit[4]
        pos1 = edit[5]
        pos2 = edit[6]
        ref = edit[11]
        alt = edit[12]
        input_str = str(chromNum) + ',' + str(pos1) + ',' + str(pos2) + ',' + ref + ',' + alt
        ptms = getPTMSFromLocation(input_str)
        print(ptms)

HTTPBadRequest: 400 : Response specification matching http status_code 400 not found for operation Operation(fetchVariantAnnotationByGenomicLocationGET). Either add a response specification for the status_code or use a `default` response.